Importing dependencies

In [ ]:
!pip install -r requirements.txt 

In [ ]:
import cv2
import numpy as np
import pytesseract
import requests
from PIL import Image
import os
import time
from discord_webhook import DiscordWebhook
import json
from mss import mss
from config import *

Processing screens with Tesseract

In [ ]:
def toggleMute(operation):
    webhook = DiscordWebhook(url=webhookURL,content=">>au {} {}".format(operation, channelID))
    response = webhook.execute()
    sleep(0.5)

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

def processDiscussion(image: complex) -> int:
    #1 = Not found, 2 = Discussion found, 3 = Voting ended found

    discussion = {"?","impestoe","who",'whos',"wino","innoosttor?","imsoster?","inostor?","imposter?","inyoostor?","iniposior?","inijposior?","impostor?","inoster?","tnrpester?","tnsester?","inraostor?","inaoster?","tnsoster?","tnpester?",'hnnsester?'}
    discussion.update(keyword_whos_imposter)
    voting = {"voting", "results","result","vetting","vartine","votingiresults","vetting)","\\n\\nvatiing","results\\n\\n","resulis\\n\\n","resuilis\\n\\n","resulis","resuilis"}
    discussion.update(keyword_voting_ended)
    raw_output = pytesseract.image_to_string(image)
    
    out = set(raw_output.strip().strip('\n').strip("\\").strip("/").lower().split(" "))

    if debug_mode:
        print(out)

    if len(out.intersection(discussion)) != 0: #if one of the keywords for discussion time is present
        print("DISCUSSION [UNMUTED]")
        toggleMute("unmute")

        return 2

    elif len(out.intersection(voting)) != 0: #if one of the keywords for ended voting is present
        print("VOTING ENDED [MUTING SOON]")

        return 3
    else:
        return 1

def processEnding(image: complex) -> bool:
    delay = 3.5 #Delay between getting role and game starting

    defeat = {"defeat","deteat","netrtorat","neffeat","netfeat","defeat\\n\\n"}
    defeat.update(keyword_defeat)
    victory = {"victory","vicory","viton"}
    victory.update(keyword_victory)
    imposter = {"imposter","impostor","tmonetor"}
    imposter.update(keyword_imposter)
    crewmate = {"crewmate"}
    crewmate.update(keyword_crewmate)
    
    raw_output = pytesseract.image_to_string(image)
    
    out = set(raw_output.strip().strip('\n').strip("\\").strip("/").lower().split(" "))
    
    if debug_mode:
        print(out)

    if len(out.intersection(defeat)) != 0: #if one of the keywords for defeat is present
        print("DEFEAT [UNMUTED]")
        toggleMute("unmute") #unmute everyone including the dead
        return True

    elif len(out.intersection(victory)) != 0: #if one of the keywords for victory is present
        print("VICTORY [UNMUTED]")
        toggleMute("unmute") #unmute everyone including the dead
        return True

    elif len(out.intersection(crewmate)) != 0: #if one of the keywords for crewmate is present
        print("YOU GOT CREWMATE [MUTING SOON]")

        time.sleep(delay + delay_start)
        toggleMute("mute") #mute
        return False

    elif len(out.intersection(imposter)) != 0: #if one of the keywords for imposter is present
        print("YOU GOT IMPOSTER [MUTING SOON]")

        time.sleep(delay + delay_start)
        toggleMute("mute") #mute
        return False
    else:
        print(".")
        return False

Grabbing Screen from monitor

In [ ]:
def returnFrame(settings: set, sct) -> complex:
    #Take image of screen
    sct_img = sct.grab(settings)
    img = Image.frombytes('RGB', (sct_img.size.width, sct_img.size.height), sct_img.rgb)
    frame = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    return frame

In [ ]:
def grabScreen(xResolution: int, yResolution: int):
    keep_grabbing = False

    settings = {"top": int(0.08 * yResolution) + adjust_y, "left":int(xResolution * 0.18) + adjust_x, "width":int(xResolution * 0.7), "height":int(0.25 * yResolution), "mon": monitor_number}

    first_time = True

    x_crop = int(xResolution * 0.18)
    y_crop = int(yResolution * 0.08)

    with mss() as sct:
        while True:
            frame = returnFrame(settings, sct)
            
            try:
                #Crop image to fit only "voting ended" and "whos the imposter?"
                cropped_frame = frame[10:(y_crop + y_extend_crop), int(x_crop/2 - x_extend_crop + 80):-int(x_crop/2 + x_extend_crop)].copy()
            
                if debug_mode:
                    cv2.imshow('Test', np.array(frame)) #output screen, for testing only
                    cv2.imshow('Test Cropped', np.array(cropped_frame)) #output screen, for testing only

                    if cv2.waitKey(25) & 0xFF == ord('q'): #Press Q on debug windows to exit
                        cv2.destroyAllWindows()
                        break

            except Exception as e:
                print(f"{e}\n[*] Looks like your x_extend_crop or y_extend_crop values are way too high")
                exit()

            if first_time:
                print("[*] Screen grab ready.\n")
                first_time = False

            #Process image
            found = processDiscussion(cropped_frame)

            if found == 1: #If discussion or voting ends found, you dont need to process ending
                processEnding(frame)
            
            elif found == 3:
                start_time = time.time()

                while True:
                    see_if_ended = returnFrame(settings, sct)
                    ended = processEnding(see_if_ended)

                    if ended:
                        break

                    elif time.time() - start_time >= 6 + delay_voting:
                        grabScreen("mute")
                        break
            else: pass

In [ ]:
try:
    x, y = int(screen_resolution.split("x")[0]), int(screen_resolution.split("x")[1])
    grabScreen(x,y)

except KeyboardInterrupt:
    print("\n[*] Stopping bot...")

except Exception as e:
    print(e)